In [11]:
import os
import glob
import time
import pantab
import credentials
import pandas as pd
from tableau_api_lib import TableauServerConnection
from snowflake.connector.pandas_tools import write_pandas
from tableau_api_lib.utils.querying import get_projects_dataframe
from snowflake.connector import connect as snowflake_connect, DictCursor

In [12]:
project_name = 'dogs_of_zurich'
snowflake_table_name = f'{project_name}_test'.upper()

In [13]:
os_path = os.getcwd()
if os_path.split('\\')[-1] == 'python':
    data_path = "\\".join(os_path.split('\\')[:-1])+"\data"
else:
    data_path = f'{os_path}\\{project_name} data'   
os.makedirs(data_path, exist_ok=True)

# Data ingestion and cleaning

In [14]:
csv_list = glob.glob(f'{data_path}/*hundehalter.csv') # Using Glob to use a wildcard for the file name

In [15]:
column_translation = { # Dictionary to rename the columns in the DF
                    "HALTER_ID": "OWNER_ID",
                    "ALTER": "AGE_GROUP",
                    "GESCHLECHT": "GENDER",
                    "STADTKREIS": "DISTRICT",
                    "RASSE1": "PRIMARY_BREED",
                    "RASSE2": "SECONDARY_BREED",
                    "GEBURTSJAHR_HUND": "DOG_YEAR_OF_BIRTH",
                    "GESCHLECHT_HUND": "DOG_GENDER",
                    "HUNDEFARBE": "DOG_COLOUR"
                    }

german_names = [german_name for german_name, english_name in column_translation.items()] # List comprehension to get only german name to filter the DF afterwards

In [25]:
full_dataset = pd.DataFrame() # Data frame to concatenate the tables

for csv_name in csv_list: # looping thru the file names
    year = csv_name.split('\\')[-1][:4] # Spliting the file path and selecting the first 4 char. to get the year
    
    data = pd.read_csv(csv_name) # Reading csv files
    data = data[german_names] # Selecting columns based on german_names
    data['DATASET_YEAR'] = int(year) # Adding the Year to the dataset
    data['UPDATE_TIME'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    
    full_dataset = pd.concat([full_dataset, data], ignore_index = True) # Union the different files
    
full_dataset.rename(columns = column_translation, inplace = True) # Renaming of columns based on column_translation

full_dataset.head()

,OWNER_ID,AGE_GROUP,GENDER,DISTRICT,PRIMARY_BREED,SECONDARY_BREED,DOG_YEAR_OF_BIRTH,DOG_GENDER,DOG_COLOUR,DATASET_YEAR,UPDATE_TIME
0,126,51-60,m,9.0,Welsh Terrier,NaN,2011,w,schwarz/braun,2015,2023-10-12 10:43:28
1,574,61-70,w,2.0,Cairn Terrier,NaN,2002,w,brindle,2015,2023-10-12 10:43:28
2,695,41-50,m,6.0,Labrador Retriever,NaN,2012,w,braun,2015,2023-10-12 10:43:28
3,893,61-70,w,7.0,Mittelschnauzer,NaN,2010,w,schwarz,2015,2023-10-12 10:43:28
4,1177,51-60,m,10.0,Shih Tzu,NaN,2011,m,schwarz/weiss,2015,2023-10-12 10:43:28


In [26]:
color_translation = {
                    "schwarz" : "black",
                    "weiss": "white",
                    "braun": "brown"
                    }

def translate_color(color_str): # Function with a list comprehension to translate all the colors regardless if they are mixed but match the color name
    translated_parts = [color_translation.get(part, part) for part in color_str.split('/')]
    return '/'.join(translated_parts)

full_dataset['DOG_COLOUR'] = full_dataset['DOG_COLOUR'].apply(translate_color)

full_dataset.head()

,OWNER_ID,AGE_GROUP,GENDER,DISTRICT,PRIMARY_BREED,SECONDARY_BREED,DOG_YEAR_OF_BIRTH,DOG_GENDER,DOG_COLOUR,DATASET_YEAR,UPDATE_TIME
0,126,51-60,m,9.0,Welsh Terrier,NaN,2011,w,black/brown,2015,2023-10-12 10:43:28
1,574,61-70,w,2.0,Cairn Terrier,NaN,2002,w,brindle,2015,2023-10-12 10:43:28
2,695,41-50,m,6.0,Labrador Retriever,NaN,2012,w,brown,2015,2023-10-12 10:43:28
3,893,61-70,w,7.0,Mittelschnauzer,NaN,2010,w,black,2015,2023-10-12 10:43:28
4,1177,51-60,m,10.0,Shih Tzu,NaN,2011,m,black/white,2015,2023-10-12 10:43:28


In [28]:
column_translation = {"HUNDERASSE" : "PRIMARY_BREED",
                      "HUNDERASSENTYP" : "BREED_TYPE"}

german_names = [german_name for german_name, english_name in column_translation.items()] # List comprehension to get only german name to filter the DF afterwards

breed_info = pd.read_csv(f'{data_path}\zuordnungstabellehunderassehundetyp.csv') # Reading csv files

breed_info = breed_info[german_names]
breed_info.rename(columns = column_translation, inplace = True) # Renaming of columns based on column_translation

breed_info.head()

,PRIMARY_BREED,BREED_TYPE
0,Affenpinscher,Kleinwüchsig
1,Afghane,Rassentypenliste I
2,Afghanischer Windhund,Rassentypenliste I
3,Airedale Terrier,Rassentypenliste I
4,Akita Inu,Rassentypenliste I


In [29]:
merged_dataset = full_dataset.merge(breed_info, on='PRIMARY_BREED', how='left')
merged_dataset.head()

,OWNER_ID,AGE_GROUP,GENDER,DISTRICT,PRIMARY_BREED,SECONDARY_BREED,DOG_YEAR_OF_BIRTH,DOG_GENDER,DOG_COLOUR,DATASET_YEAR,UPDATE_TIME,BREED_TYPE
0,126,51-60,m,9.0,Welsh Terrier,NaN,2011,w,black/brown,2015,2023-10-12 10:43:28,Kleinwüchsig
1,574,61-70,w,2.0,Cairn Terrier,NaN,2002,w,brindle,2015,2023-10-12 10:43:28,Kleinwüchsig
2,695,41-50,m,6.0,Labrador Retriever,NaN,2012,w,brown,2015,2023-10-12 10:43:28,Rassentypenliste I
3,893,61-70,w,7.0,Mittelschnauzer,NaN,2010,w,black,2015,2023-10-12 10:43:28,Rassentypenliste I
4,1177,51-60,m,10.0,Shih Tzu,NaN,2011,m,black/white,2015,2023-10-12 10:43:28,Kleinwüchsig


# Data Output

In [31]:
output_data_path = f'{data_path}//output'
os.makedirs(output_data_path, exist_ok=True)

merged_dataset.to_csv(f'{output_data_path}//{project_name}.csv', header=True, index=False, encoding='utf-8-sig') # Save to csv
merged_dataset.to_json(f'{output_data_path}//{project_name}.json', orient='records', lines=True) # Save to Json
pantab.frame_to_hyper(merged_dataset,f'{output_data_path}//{project_name}.hyper',  table = f'{project_name}') # Save to Hyper

### Publish to Tableau server

In [ ]:
# Publish to tableau
ts_config = {
            'my_env':
                    {
                    'server': credentials.tableau_server,
                    'api_version': '3.19',
                    'personal_access_token_name': credentials.tableau_token_name,
                    'personal_access_token_secret': credentials.tableau_token_secret,
                    'site_name': credentials.tableau_site_name,
                    'site_url': credentials.tableau_site_url
                    }
            }
conn = TableauServerConnection(ts_config, env='my_env')

# Sign in to tableau server.
conn.sign_in()
print (f"Logged in to {credentials.tableau_server}")

projects_df = get_projects_dataframe(conn)
project_id_list = projects_df[['name', 'id']]
    
project_id = project_id_list.loc[project_id_list['name'] == 'Sandbox', 'id'].iloc[0]

# Publishing to the server
response = conn.publish_data_source(
                    project_id = project_id,
                    datasource_file_path = f'{output_data_path}//{project_name}.hyper',
                    datasource_name = f'{project_name}',
                    datasource_description = f'{project_name} ds desc')

if response.status_code == 201:
        print(f'{project_name} Datasource successfully published.')
else:
        f'{project_name} Datasource Failed to publish'
        
conn.sign_out()
print (f"Logged Out from {credentials.tableau_server}")

### Publish to snowflake

In [40]:
# Connect to snowflake
try:
    # Establish a connection
    connection = snowflake_connect(
                                    user = credentials.snowflake_user,
                                    password = credentials.snowflake_password,
                                    account = credentials.snowflake_account,
                                    database = 'TEST_DB',
                                    warehouse = 'COMPUTE_WH',
                                    schema = 'TEST_SCHEMA'
                                    )
    print (f"Logged in to Snowflake")
    
    # Create a cursor
    cursor = connection.cursor()

except Exception as e:
    print(f"Error: {str(e)}")

Logged in to Snowflake


In [35]:
# Creating a table in snowflake
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS {snowflake_table_name} (
    OWNER_ID            INT,
    AGE_GROUP           VARCHAR(10),  
    GENDER              VARCHAR(1),      
    DISTRICT            FLOAT,
    PRIMARY_BREED       VARCHAR(50),  
    SECONDARY_BREED     VARCHAR(50),  
    DOG_YEAR_OF_BIRTH   SMALLINT,
    DOG_GENDER          VARCHAR(1),  
    DOG_COLOUR          VARCHAR(30),
    DATASET_YEAR        SMALLINT,
    UPDATE_TIME         TIMESTAMP,
    BREED_TYPE          VARCHAR(50)
)
'''
)
# Pandas to snowflake
try:
    write_pandas(connection, merged_dataset, table_name = snowflake_table_name)
    print (f"{snowflake_table_name} table succesfully published to snowflake")
    
except Exception as e:
    print(f"Could not publish the {snowflake_table_name} table to snowflake \n Error: {str(e)}")

DOGS_OF_ZURICH_TEST table succesfully published to snowflake


In [41]:
# Snowflake to pandas
cursor = connection.cursor(DictCursor)
cursor.execute(f'SELECT * FROM {snowflake_table_name}')
data_fetch = pd.DataFrame(cursor.fetchall())
data_fetch.head()

,OWNER_ID,AGE_GROUP,GENDER,DISTRICT,PRIMARY_BREED,SECONDARY_BREED,DOG_YEAR_OF_BIRTH,DOG_GENDER,DOG_COLOUR,DATASET_YEAR,UPDATE_TIME,BREED_TYPE
0,126,51-60,m,9.0,Welsh Terrier,None,2011,w,black/brown,2015,2023-10-12 09:55:30,Kleinwüchsig
1,574,61-70,w,2.0,Cairn Terrier,None,2002,w,brindle,2015,2023-10-12 09:55:30,Kleinwüchsig
2,695,41-50,m,6.0,Labrador Retriever,None,2012,w,brown,2015,2023-10-12 09:55:30,Rassentypenliste I
3,893,61-70,w,7.0,Mittelschnauzer,None,2010,w,black,2015,2023-10-12 09:55:30,Rassentypenliste I
4,1177,51-60,m,10.0,Shih Tzu,None,2011,m,black/white,2015,2023-10-12 09:55:30,Kleinwüchsig


In [42]:
# Close the cursor and connection
cursor.close()
connection.close()
print (f"Logged Out from Snowflake")

Logged Out from Snowflake
